In [2]:
import sys
from concurrent import futures
import numpy as np
from pyspark.sql import SparkSession
from datetime import datetime,timedelta
import pytz

# print(datetime.now(pytz.timezone("Asia/Shanghai")).strftime( '%Y-%m-%d'))
today=(datetime.now()+ timedelta(-1)).strftime( '%Y-%m-%d')
yesterday=(datetime.now(pytz.timezone("Asia/Shanghai"))+ timedelta(-1)).strftime( '%Y-%m-%d')
# ,"banda_stream_etl":"1"
tablemap={
    "banda":{
         "ap_social_media_summary_rpt_weekly":"""
             with track_log as 
                (
                select 
                page_id
                ,action_type
                ,user_uuid
                ,widget_id
                ,view_path
                ,date(start_time+interval'7'hour) as data_date
                FROM `ddb_event_track_s3`.`id_adapundi_event_track_log`
                WHERE partition_0 = '2021'
                and   partition_1>='07'
                union all 
                select 
                page_id
                ,action_type
                ,user_uuid
                ,widget_id
                ,view_path
                ,date(start_time+interval'7'hour) as data_date
                FROM `ddb_event_track_s3`.`id_adapundi_event_track_log`
                WHERE partition_0 > '2021'
                ),
                track_log_app as 
                (
                SELECT  *,'fb' as app_name
                FROM track_log
                WHERE   widget_id in('ll_setting_facebook','f_call_facebook') or view_path='AbountScreen-0-5[1]-1'
                union all 
                SELECT  *,'ins' as app_name
                FROM track_log
                WHERE   widget_id in('ll_setting_ins','f_call_instagram') or view_path='AbountScreen-0-5[1]-1[1]'
                union all 
                SELECT  *,'twitter' as app_name
                FROM track_log
                WHERE   widget_id in('f_call_twitter') or view_path='AbountScreen-0-5[1]-1[2]'
                )

                -- 新建表-每周一刷一次-增量表，历史数据保存
                -- banda_rpt_mid.ap_social_media_summary_rpt_weekly
                SELECT  DATE(now()+INTERVAL '-1'day+interval'7'hour)                                                 AS data_date
                       ,a.app_name
                       ,COUNT(distinct if(DATE(now()+INTERVAL '-1'day+interval'7'hour)>=data_date,a.user_uuid,null)) AS summary
                       ,COUNT(distinct if(min_date=DATE(now()+INTERVAL '-1'day+interval'7'hour),b.user_uuid,null))   AS new
                FROM track_log_app a
                LEFT JOIN
                (
                    SELECT  user_uuid
                           ,app_name
                           ,MIN(data_date) min_date
                    FROM track_log_app
                    group by user_uuid,app_name
                ) b
                ON a.user_uuid=b.user_uuid AND a.app_name=b.app_name
                GROUP BY  a.app_name """  ,
        
    }
   
}
def make_col_temptable(tableType,tableNm):
    sql=tablemap[tableType][tableNm]
    tablepath="s3://rupiahplus-data-warehouse/etl/"+tableType+"/report_temp_table/"+tableNm
    spark.sql(sql).write.mode("append").orc(tablepath)
#     builtins
# 
if __name__ == "builtins":
    spark = SparkSession\
        .builder\
        .appName("Python Demo")\
        .config("hive.metastore.client.factory.class", "com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory") \
        .enableHiveSupport()\
        .getOrCreate()
#     spark.conf.set("spark.scheduler.mode","FAIR")
    spark.conf.set("spark.sql.shuffle.partitions",20)
    for tableType in tablemap:
#         databasesql="show tables in "+databaseName
#         tables=spark.sql(databasesql)
#         tablelist=tables.collect();
        executor=None
        with futures.ThreadPoolExecutor(max_workers=1) as executor:
            futures_result=futures.wait([executor.submit(make_col_temptable,tableType, tableNm) for tableNm in tablemap[tableType]])
            for  future in futures_result[0]:
                print(future.result())
           


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

None

In [5]:
spark = SparkSession\
    .builder\
    .appName("Python Demo")\
    .config("hive.metastore.client.factory.class", "com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory") \
    .enableHiveSupport()\
    .getOrCreate()
spark.sql("""
with track_log as 
(
select 
page_id
,action_type
,user_uuid
,widget_id
,view_path
,concat(partition_0,'-',partition_1,'-',partition_2) as data_date
FROM `ddb_event_track_s3`.`id_adapundi_event_track_log`
WHERE partition_0 = '2021'
and   partition_1>='07'
union all 
select 
page_id
,action_type
,user_uuid
,widget_id
,view_path
,concat(partition_0,'-',partition_1,'-',partition_2) as data_date
FROM `ddb_event_track_s3`.`id_adapundi_event_track_log`
WHERE partition_0 > '2021'
)
SELECT  'UserCenter'              AS last_page
       ,'UserCenter'              AS page_id
       ,data_date
       ,COUNT(distinct user_uuid) AS enter_page_user
FROM track_log
WHERE page_id ='UserCenter'
AND action_type = 'page_entrance_event'
GROUP BY  data_date

union all
SELECT  'UserCenter'              AS last_page
       ,'AboutAdaPundi'        AS page_id
       ,data_date
       ,COUNT(distinct user_uuid) AS enter_page_user
FROM track_log
WHERE  widget_id='ll_about'
GROUP BY  data_date

union all
SELECT  'HomePageRepayment'              AS last_page
       ,'FraudPage'        AS page_id
       ,data_date
       ,COUNT(distinct user_uuid) AS enter_page_user
FROM track_log
WHERE  widget_id='homeRepay_fraud_layout'
GROUP BY  data_date

""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+----------+---------------+
| last_page|   page_id| data_date|enter_page_user|
+----------+----------+----------+---------------+
|UserCenter|UserCenter|2021-07-18|          15320|
|UserCenter|UserCenter|2021-07-13|          17410|
|UserCenter|UserCenter|2021-07-20|          14804|
|UserCenter|UserCenter|2021-08-07|          14835|
|UserCenter|UserCenter|2021-07-15|          17571|
|UserCenter|UserCenter|2021-07-04|          14943|
|UserCenter|UserCenter|2021-07-19|          18100|
|UserCenter|UserCenter|2021-08-06|          16942|
|UserCenter|UserCenter|2021-07-07|          17328|
|UserCenter|UserCenter|2021-07-16|          17547|
|UserCenter|UserCenter|2021-07-11|          15808|
|UserCenter|UserCenter|2021-08-04|          17515|
|UserCenter|UserCenter|2021-07-24|          17756|
|UserCenter|UserCenter|2021-07-08|          17339|
|UserCenter|UserCenter|2021-07-27|          17407|
|UserCenter|UserCenter|2021-08-08|          14231|
|UserCenter|UserCenter|2021-07-

In [2]:
spark.sql("""
-- banda_rpt_mid.ap_social_media_rpt_daily
-- 每周一全量刷新一次
with track_log as 
(
select 
page_id
,action_type
,user_uuid
,widget_id
,view_path
,concat(partition_0,'-',partition_1,'-',partition_2) as data_date
FROM `ddb_event_track_s3`.`id_adapundi_event_track_log`
WHERE partition_0 = '2021'
and   partition_1>='07'
union all 
select 
page_id
,action_type
,user_uuid
,widget_id
,view_path
,concat(partition_0,'-',partition_1,'-',partition_2) as data_date
FROM `ddb_event_track_s3`.`id_adapundi_event_track_log`
WHERE partition_0 > '2021'
),
DAU as 
(
SELECT  data_date
       ,COUNT(distinct user_uuid) AS APP_DAU
FROM track_log
GROUP BY  data_date
),
impression as 
(
SELECT  data_date
       ,page_id as last_page
       ,COUNT(distinct user_uuid) AS impression_user
FROM track_log
WHERE page_id IN ('UserCenter', 'HomePageRepayment')
AND action_type = 'page_entrance_event'
GROUP BY  data_date
         ,page_id
),
enter_page as
(
SELECT  'UserCenter'              AS last_page
       ,'UserCenter'              AS page_id
       ,data_date as data_date
       ,COUNT(distinct user_uuid) AS enter_page_user
FROM track_log
WHERE page_id ='UserCenter'
AND action_type = 'page_entrance_event'
GROUP BY  data_date

union all
SELECT  'UserCenter'              AS last_page
       ,'AboutAdaPundi'        AS page_id
       ,data_date as data_date
       ,COUNT(distinct user_uuid) AS enter_page_user
FROM track_log
WHERE  widget_id='ll_about'
GROUP BY  data_date

union all
SELECT  'HomePageRepayment'              AS last_page
       ,'FraudPage'        AS page_id
       ,data_date as data_date
       ,COUNT(distinct user_uuid) AS enter_page_user
FROM track_log
WHERE  widget_id='homeRepay_fraud_layout'
GROUP BY  data_date
),
fb_click as
(
SELECT  'UserCenter'              AS last_page
       ,'UserCenter'        AS page_id
       ,data_date as data_date
       ,COUNT(distinct user_uuid) AS FB_click_user
FROM track_log
WHERE   widget_id='ll_setting_facebook'
GROUP BY  data_date

union all 
SELECT  'UserCenter'              AS last_page
       ,'AboutAdaPundi'        AS page_id
       ,data_date as data_date
       ,COUNT(distinct user_uuid) AS FB_click_user
FROM track_log
WHERE   view_path='AbountScreen-0-5[1]-1'
GROUP BY  data_date

union all 
SELECT  'HomePageRepayment'              AS last_page
       ,'FraudPage'        AS page_id
       ,data_date as data_date
       ,COUNT(distinct user_uuid) AS FB_click_user
FROM track_log
WHERE  widget_id='f_call_facebook'
GROUP BY  data_date
),
ins_click as
(
SELECT  'UserCenter'              AS last_page
       ,'UserCenter'        AS page_id
       ,data_date as data_date
       ,COUNT(distinct user_uuid) AS Ins_click_user
FROM track_log
WHERE   widget_id='ll_setting_ins'
GROUP BY  data_date

union all 
SELECT  'UserCenter'              AS last_page
       ,'AboutAdaPundi'        AS page_id
       ,data_date as data_date
       ,COUNT(distinct user_uuid) AS Ins_click_user
FROM track_log
WHERE   view_path='AbountScreen-0-5[1]-1[1]'
GROUP BY  data_date

union all 
SELECT  'HomePageRepayment'              AS last_page
       ,'FraudPage'        AS page_id
       ,data_date as data_date
       ,COUNT(distinct user_uuid) AS Ins_click_user
FROM track_log
WHERE   widget_id='f_call_instagram'
GROUP BY  data_date
),
twitter_click as
(
SELECT  'UserCenter'              AS last_page
       ,'AboutAdaPundi'        AS page_id
       ,data_date as data_date
       ,COUNT(distinct user_uuid) AS Twitter_click_user
FROM track_log
WHERE view_path='AbountScreen-0-5[1]-1[2]'
GROUP BY  data_date

union all 
SELECT  'HomePageRepayment'              AS last_page
       ,'FraudPage'        AS page_id
       ,data_date as data_date
       ,COUNT(distinct user_uuid) AS Twitter_click_user
FROM track_log
WHERE   widget_id='f_call_twitter'
GROUP BY  data_date
)

select 
a.*
,c.last_page as last_page
,c.page_id as page_id
,b.impression_user as impression_user
,c.enter_page_user as enter_page_user
,coalesce(d.FB_click_user,0) as FB_click_user
,coalesce(e.Ins_click_user,0) as Ins_click_user
,coalesce(f.Twitter_click_user,0) as Twitter_click_user
from DAU a
left join impression     b on a.data_date=b.data_date
left join enter_page     c on b.data_date=c.data_date  and b.last_page=c.last_page
left join fb_click       d on c.data_date=d.data_date  and c.last_page=d.last_page and c.page_id=d.page_id
left join ins_click      e on c.data_date=e.data_date  and c.last_page=e.last_page and c.page_id=e.page_id
left join twitter_click  f on c.data_date=f.data_date  and c.last_page=f.last_page and c.page_id=f.page_id



""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '400' from https://10.3.0.24:18888/sessions/3/statements/2 with error payload: {"msg":"requirement failed: Session isn't active."}


In [ ]:
spark.sql("""with track_log as 
(
select 
page_id
,action_type
,user_uuid
,widget_id
,view_path
,date(start_time+interval'7'hour) as data_date
FROM `ddb_event_track_s3`.`id_adapundi_event_track_log`
WHERE partition_0 = '2021'
and   partition_1>='07'
union all 
select 
page_id
,action_type
,user_uuid
,widget_id
,view_path
,date(start_time+interval'7'hour) as data_date
FROM `ddb_event_track_s3`.`id_adapundi_event_track_log`
WHERE partition_0 > '2021'
),
track_log_app as 
(
SELECT  *,'fb' as app_name
FROM track_log
WHERE   widget_id in('ll_setting_facebook','f_call_facebook') or view_path='AbountScreen-0-5[1]-1'
union all 
SELECT  *,'ins' as app_name
FROM track_log
WHERE   widget_id in('ll_setting_ins','f_call_instagram') or view_path='AbountScreen-0-5[1]-1[1]'
union all 
SELECT  *,'twitter' as app_name
FROM track_log
WHERE   widget_id in('f_call_twitter') or view_path='AbountScreen-0-5[1]-1[2]'
)

-- 新建表-每周一刷一次-增量表，历史数据保存
-- banda_rpt_mid.ap_social_media_summary_rpt_weekly
SELECT  DATE(now()+INTERVAL '-1'day+interval'7'hour)                                                 AS data_date
       ,a.app_name
       ,COUNT(distinct if(DATE(now()+INTERVAL '-1'day+interval'7'hour)>=data_date,a.user_uuid,null)) AS summary
       ,COUNT(distinct if(min_date=DATE(now()+INTERVAL '-1'day+interval'7'hour),b.user_uuid,null))   AS new
FROM track_log_app a
LEFT JOIN
(
	SELECT  user_uuid
	       ,app_name
	       ,MIN(data_date) min_date
	FROM track_log_app
    group by user_uuid,app_name
) b
ON a.user_uuid=b.user_uuid AND a.app_name=b.app_name
GROUP BY  a.app_name





""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

KeyboardInterrupt: 